In [1]:
import os
import torch
import shutil
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

TARGET_MODEL = "/home/student/Downloads/copycat_fer_project/target_resnet18_3class.pth"
NPDD_DIR = "/home/student/Downloads/copycat_fer_project/npdd_images"
OUT_DIR = "/home/student/Downloads/copycat_fer_project/npdd_sl"

CLASS_NAMES = ["angry", "happy", "neutral"]

# Create output folders
os.makedirs(OUT_DIR, exist_ok=True)
for c in CLASS_NAMES:
    os.makedirs(os.path.join(OUT_DIR, c), exist_ok=True)

# Transform (same as target)
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Load target model
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 3)
model.load_state_dict(torch.load(TARGET_MODEL, map_location=DEVICE))
model.to(DEVICE)
model.eval()

count = 0

with torch.no_grad():
    for img_name in os.listdir(NPDD_DIR):
        img_path = os.path.join(NPDD_DIR, img_name)

        try:
            img = Image.open(img_path).convert("RGB")
        except:
            continue

        x = transform(img).unsqueeze(0).to(DEVICE)
        out = model(x)
        pred = out.argmax(1).item()

        label = CLASS_NAMES[pred]
        shutil.copy(img_path, os.path.join(OUT_DIR, label, img_name))
        count += 1

print("NPDD-SL fake dataset generated successfully.")
print("Total images processed:", count)


NPDD-SL fake dataset generated successfully.
Total images processed: 7512
